In [2]:
!pip install spacy

  Using cached spacy-3.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
  Using cached murmurhash-1.0.7-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached pydantic-1.8.2-cp39-cp39-manylinux2014_x86_64.whl (11.3 MB)
  Using cached catalogue-2.0.7-py3-none-any.whl (17 kB)
  Using cached srsly-2.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (457 kB)
  Using cached blis-0.7.7-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.9 MB)
  Using cached pathy-0.6.1-py3-none-any.whl (42 kB)
  Using cached wasabi-0.9.1-py3-none-any.whl (26 kB)
  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
     |████████████████████████████████| 668 kB 9.6 MB/s            
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached typer-0.4.1-py3-none-any.whl (27 kB)
  Using cached spacy_loggers-1.0.2-py3-none-any.whl (7.2 kB)
  Using cached preshed-3.0.6-cp39-cp39-manylinux_

In [3]:
!python -m spacy download fr_core_news_md -q

✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [4]:
!pip install Unidecode

  Using cached Unidecode-1.3.4-py3-none-any.whl (235 kB)


In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fr_core_news_md
import pickle as pk

REVIEWS LOOX

In [6]:
df_reviews = pd.read_csv('reviews_12052022.csv')

In [7]:
df_reviews.columns

Index(['id', 'status', 'rating', 'email', 'img', 'nickname', 'review', 'date',
       'productId', 'handle', 'variant', 'verified_purchase', 'orderId',
       'reply', 'replied_at'],
      dtype='object')

In [ ]:
# Données cachés pour des raisons de confidentialités

In [8]:
df_reviews.head(0)

,id,status,rating,email,img,nickname,review,date,productId,handle,variant,verified_purchase,orderId,reply,replied_at


DONNEES D'ENTRAINEMENT

In [8]:
# Suppression de toutes les colonnes sauf rating ,reviews et date

In [9]:
df_reviews = df_reviews[["rating", "review", "date"]].copy()

In [10]:
df_reviews.head(0)

,rating,review,date


PREPROCESSING

In [11]:
# Initialisation en français
nlp = fr_core_news_md.load()

In [12]:
### Calcul des lemmes et pos via spacy

In [13]:
df_reviews["token_spacy"] = df_reviews.review.map(lambda s: nlp(s))

#df.tokens
tokens = df_reviews.token_spacy.tolist()
tokens = [(i.lemma_, i.pos_, i.text) for s in tokens for i in s]
tokens = pd.DataFrame(tokens, columns = ['rating','review','date'])
#tokens = tokens.groupby('lemma').agg(list)

In [14]:
df_reviews.head(0)

,rating,review,date,token_spacy


In [15]:
#df_reviews.token_spacy[6]

In [16]:
for tok in df_reviews.token_spacy.iloc[10]:
    print(tok.lemma_, tok.pos_, tok.text)

j’ PRON J’
avoir AUX ai
pas ADV pas
l’ DET l’
habitude NOUN habitude
de ADP de
porter VERB porter
ce DET ce
type NOUN type
de ADP de
culotte NOUN culotte
mais CCONJ mais
super VERB super
à ADP à
l’ DET l’
aise NOUN aise
dedans PROPN dedans
, PUNCT ,
merci NOUN merci
. PUNCT .


In [17]:
def lemmas_filtered_on_pos(tokens, pos_list=["VERB", "NOUN", "ADJ", "ADV"]):
    return " ".join([t.lemma_ for t in tokens if t.pos_ in pos_list])

In [18]:
df_reviews["review_clean"] = df_reviews.token_spacy.map(lemmas_filtered_on_pos)

In [19]:
tokens_ = df_reviews[df_reviews.review.str.contains("pas")].token_spacy.iloc[0]


for x in tokens_:
    print(x.text, x.pos_)

J’ PRON
ai AUX
pas ADV
l’ DET
habitude NOUN
de ADP
porter VERB
ce DET
type NOUN
de ADP
culotte NOUN
mais CCONJ
super VERB
à ADP
l’ DET
aise NOUN
dedans PROPN
, PUNCT
merci NOUN
. PUNCT


````
REMPLACEMENT DES 'NOTES' 5 ET 4 PAR "1" ET 3,2,1 PAR "0" 
````

df_reviews["new_rating"] = (df_reviews["rating"] >=4)*1   # *1 pour passer du True et False à 0 et 1
df_reviews

In [20]:
df_reviews.to_pickle('df_reviews_lemmatiz_filtre.plk')